In [3]:
import sys
sys.path.append('../')
from matformer.tensors_dataclasses import TensorDC, NormalTensor, PaddedTensor, UnpaddedTensor
import torch
from matformer.matformer_registry import registry


In [4]:
# Dimensioni ipotetiche
q_dim=512
k_dim=512
v_dim=512
seqlen=1024
batch_size=32
n_heads=8
bias=False
assert q_dim%n_heads==0
head_size=q_dim//n_heads


In [5]:
# Tensori di prova
query_input_batched=torch.randn(batch_size,seqlen,q_dim)
query_input_unbatched=torch.randn(batch_size*seqlen,q_dim)
print(f"Batched: {query_input_batched.shape}, Unbatched: {query_input_unbatched.shape}")


Batched: torch.Size([32, 1024, 512]), Unbatched: torch.Size([32768, 512])


In [22]:
## Proiezioni iniziali (a q_dim o 3*q_dim)
#if not is_cross_attention or self.qkv_samedim:
#self.packed_proj = nn.Linear(self.q_dim, 3 * q_dim, bias=bias)
packed_proj=registry.create('linear','linear',in_features=q_dim, out_features=3*q_dim,bias=bias)
#else:
q_proj=registry.create('linear','linear',in_features=q_dim, out_features=q_dim,bias=bias)
k_proj=registry.create('linear','linear',in_features=k_dim, out_features=q_dim,bias=bias)
v_proj=registry.create('linear','linear',in_features=v_dim, out_features=q_dim,bias=bias)
#self.q_proj = nn.Linear(q_dim, q_dim, bias=bias)
#self.k_proj = nn.Linear(k_dim, q_dim, bias=bias)
#self.v_proj = nn.Linear(v_dim, q_dim, bias=bias)

In [9]:
qkv_projected_batched=NormalTensor(tensor=packed_proj(query_input_batched),tensor_order='?S(3*D)')
#qkv_projected_unbatched=packed_proj(query_input_unbatched)
#print(f"(QKV PACKED PROJECTION): Batched: {qkv_projected_batched.shape}, Unbatched: {qkv_projected_unbatched.shape}")
#qkv_projected_shape='?S(3*D)'

q_batched=NormalTensor(tensor=q_proj(query_input_batched),tensor_order='?SD')
#q_unbatched=q_proj(query_input_unbatched)
#print(f"(Q NORMAL PROJECTION): Batched: {q_batched.shape}, Unbatched: {q_unbatched.shape}")
#q_projected_shape='?SD'

In [11]:
q_batched.tensor_order

'?SD'

In [18]:
#qkv_projected_heads_batched=qkv_projected_batched.unflatten(-1,[3,n_heads,head_size])
#qkv_projected_heads_unbatched=qkv_projected_unbatched.unflatten(-1,[3,n_heads,head_size])
q_projected_heads_batched=q_batched.tensor.unflatten(-1,[n_heads,head_size])
#q_projected_heads_unbatched=q_unbatched.unflatten(-1,[n_heads,head_size])
print(f"Packed: batched: {qkv_projected_heads_batched.shape} unbatched: {qkv_projected_heads_unbatched.shape}")
print(f"Normal: batched: {q_projected_heads_batched.shape} unbatched: {q_projected_heads_unbatched.shape}")


NameError: name 'qkv_projected_heads_batched' is not defined

In [ ]:
qkv_heads_shape='?S3HD'
q_heads_shape='?SHD'


In [ ]:
q=NormalTensor(tensor=q_projected_heads_batched, tensor_order='?SHD')

In [ ]:

def transpose_for_kernel(a,b,identifier):
    print(f"Originale: {identifier}")
    identifier=list(identifier)
    c=identifier[a]
    identifier[a]=identifier[b]
    identifier[b]=c
    return "".join(identifier)
    
    

In [ ]:
transpose(1,2,q_heads_shape)

In [ ]:
q_projected_heads_batched.shape

In [ ]:
prova=q_projected_heads_batched.transpose(1,2)
q_projected_heads_batched.shape
prova.shape

In [ ]:
# Heads creation
            t = qkv_projected.tensor.unflatten(-1, [3, self.nheads, self.head_dim])
            # Detect whether batch dim exists (padded / normal) or not (unpadded represented as (S, 3, H, Hd))
            if t.dim() == 5:
                order = 'S3HD'
            elif t.dim() == 4:
                order = 'S3HD'
            else:
                raise NotImplementedError(f"Unsupported packed qkv tensor shape: {tuple(t.shape)}")
            qkv_projected = NormalTensor(tensor=t, extra_attributes={'tensor_order': order})
        else:
            q = NormalTensor(tensor=q.tensor.unflatten(-1, [self.nheads, self.head_dim]).transpose(1, 2), extra_attributes={'tensor_order': 'HSD'})

In [13]:
'?SHD'.replace('?','')


'SHD'

In [12]:
'BSHD'.replace('B','')

'SHD'

In [16]:
a='BSHD'
a.replace('B','').replace('?','')

'SHD'

In [19]:
q_projected_heads_batched.shape #?SHD

torch.Size([32, 1024, 8, 64])

In [25]:
# Come ripaccare le tre rappresentazioni separate?
repacked=torch.cat([q_projected_heads_batched, q_projected_heads_batched, q_projected_heads_batched], dim=-1)

In [24]:
repacked.shape

torch.Size([32, 1024, 24, 64])